In [1]:
import xarray as xr
import rioxarray as rio
from utils import *
from typing import Tuple
from glob import glob
from tqdm import tqdm

# Preform a sensitivity analysis to remote sensing data saturation at high biomass densities

## 1. Introduction

One possible bias in remote sensing data is the saturation of the signal at high biomass densities. This means that in dense forests, we will not be able to detect small but persistent growth in living biomass. 

To test the impact of such a bias, we perform a sensitivity analysis according to the method used in [Xu et al. (2021)](https://www.science.org/doi/10.1126/sciadv.abe9829). We apply this method to gridded datasets that report absolute biomass densities. These are:
1. Xu et al.

2. Liu et al.

3. Chen et al.

4. L-VOD data

### 1.1 Methodology summary

The approach applies a biomass density cutoff to define a "dense forest", and then applies a constant growth rate to the biomass density in these forests. 

The growth rates are taken from Table S4 of [Xu et al. (2021)](https://www.science.org/doi/10.1126/sciadv.abe9829) and are defined for different ecosystems.

To take into account disturbance in dense forests, the method calculates the area impacted by disturbance in dense forests per year. The growth rates are applied only to areas which have not been disturbed. 

We calculate the fraction of disturbed area using the following approach. Xu et al. report the total emissions from deforestation $E_C$, forest degradation $E_{FD}$, and fire in forests and nonforests $E_{Fire}$. As reported in Xu et al., the way these emissions are calculated is (for example for forest degradation):

$$E_{FD_i}=PDA_i \times C_i \times f_D$$

Where $PDA_i$ is the proportion of disturbed area in grid cell $i$, $C_i$ is the carbon density in grid cell $i$, and $f_D$ is the fraction of carbon loss per grid cell due to forest degradation.

This means that we can calculate from the emissions $E_{FD_i}$ the proportion of disturbed area in each grid cell $PDA_i$ using the following equation:
$$PDA_i = \frac{E_{FD_i}}{C_i \times f_D}$$

This is an example for forest degregation, but the same holds for PCA and PBA.

## 2. Load data

In [2]:
# biomass stocks data
xu_data = rio.open_rasterio('../data/biomass/xu_et_al_2021/test10a_cd_ab_pred_corr_2000_2019_v2.tif')
xu_data['time'] = xu_data.time.dt.year

# emissions from disturbance data
fire_data = rio.open_rasterio('../data/land_cover/xu_et_al_2021/fireforest_emission_0119_v2_inTg.tif',masked=True).rename({'band':'time'})
fire_nonforest_data = rio.open_rasterio('../data/land_cover/xu_et_al_2021/firenonforest_emission_0119_v2_inTg.tif',masked=True).rename({'band':'time'})
deforestation_data = rio.open_rasterio('../data/land_cover/xu_et_al_2021/deforestation_emission_0119_v2_inTg.tif',masked=True).rename({'band':'time'})
degradation_data = rio.open_rasterio('../data/land_cover/xu_et_al_2021/degradation_emission_0119_v2_inTg.tif',masked=True).rename({'band':'time'})

# region definitions for growth rates
xu_regions = rio.open_rasterio('../data/regions_data/xu_et_al_2021/global_ecoregions.tif').sel(band=1)

# replace regions 0,15, and 16 with NaN and convert values to start from 0
xu_regions = xr.where(xu_regions.isin([0,15,16]),np.nan,xu_regions-101)
xu_regions.rio.write_crs('EPSG:4326',inplace=True);


# growth rate data for mature forests
growth_rates = pd.read_csv('../data/biomass/xu_et_al_2021/table_s4.csv')

# set up a map of growth rates
growth_rate_map = xr.zeros_like(xu_data[0,:,:])

# set growth rates for each region
regional_growth_rate = -1*np.concatenate([growth_rates.set_index('code')['Growth rate [Mg/ha/yr]'].values,np.array([np.nan])])

# fill in the values in the growth rate map based on the regional_growth_rate indices
growth_rate_map.data = regional_growth_rate[xu_regions.fillna(growth_rates.shape[0]).values.astype(int)]

## 3. Define functions for analysis

In [3]:
def analyze_dataset_old(ds:xr.DataArray, growth_rate_map:xr.DataArray, disturbances:xr.DataArray)->Tuple[xr.DataArray,xr.DataArray]:
    """
    This function calculates the total biomass stocks assuming that "dense forests" (defined according to two thresholds - 100 and 200 Mg/ha) are growing at a constant rate.
    The growth rates are defined in the `growth_rate_map` variable. 
    From the base growth rate, we remove area that was affected by disturbance, defined by the `disturbances` varaible.
    
    Parameters:
    ds (xr.DataArray): the dataset containing the biomass stocks
    growth_rate_map (xr.DataArray): the map of growth rates
    disturbances (xr.DataArray): the map of disturbances

    Returns:
    (xr.DataArray,xr.DataArray): the biomass stocks in dense forests for the 100 Mg/ha and 200 Mg/ha thresholds
    """
    

    # define the mask for the 100 and 200 Mg/ha thresholds for "dense forests"
    mask_200 = ds.where(ds.sum(dim='landcover')>200)
    mask_100 = ds.where(ds.sum(dim='landcover')>100)

    # define the base map for the growth rates
    base_map = xr.zeros_like(ds.sel(time=ds['time'].values[0]))

    # reproject the growth rate map and disturbances to match the dataset
    gm = growth_rate_map.rio.reproject_match(ds)
    dist_map = disturbances.rio.reproject_match(ds)

    # fill in missing years in the disturbance map
    missing_years = list(set(ds['time'].values).difference(set(dist_map['time'].values)))
    if len(missing_years)>0:
        dist_map = xr.concat([dist_map,dist_map.interp(time=missing_years)],dim='time').sortby('time')
        dist_map = dist_map.bfill('time')

    # calculate the biomass accumulated in undisturbed dense forests
    intact_changes = xr.concat([base_map+gm*i for i in (ds['time']-ds['time'][0]).values],dim='time').transpose('landcover','time','y','x')
    intact_changes['time'] = ds['time']

    # apply the growth rates to the forest and nonforest fractions of the map based on their relative biomass contribution
    intact_changes = intact_changes*(ds/ds.sum(dim='landcover'))

    # remove from the "dense forests" the area fraction lost to disturbances
    intact_changes_disturbances = intact_changes.copy()
    intact_changes_disturbances[:,1:,:,:] = intact_changes_disturbances[:,1:,:,:]*(1-dist_map.fillna(0))

    # calculate the "dense forest" biomass using the stocks from the first year and the accumulated biomass
    mask_h200_stocks = (intact_changes_disturbances + mask_200.sel(time=mask_200['time'].values[0]))
    mask_h100_stocks = (intact_changes_disturbances + mask_100.sel(time=mask_100['time'].values[0]))

    # define the "non dense forest" biomass
    mask_l200_stocks = ds.where(ds.sel(time=ds['time'].values[0]).sum(dim='landcover')<200)
    mask_l100_stocks = ds.where(ds.sel(time=ds['time'].values[0]).sum(dim='landcover')<100)

    # sum the "dense" and "non dense" forest biomass
    merged_100 = (mask_h100_stocks.fillna(0) + mask_l100_stocks.fillna(0)) + (ds-ds)
    merged_200 = (mask_h200_stocks.fillna(0) + mask_l200_stocks.fillna(0)) + (ds-ds)

    
    return merged_100,merged_200

In [4]:
def analyze_dataset(ds:xr.DataArray, growth_rate_map:xr.DataArray, disturbances:xr.DataArray)->Tuple[xr.DataArray,xr.DataArray]:
    """
    This function calculates the total biomass stocks assuming that "dense forests" (defined according to two thresholds - 100 and 200 Mg/ha) are growing at a constant rate.
    The growth rates are defined in the `growth_rate_map` variable. 
    From the base growth rate, we remove area that was affected by disturbance, defined by the `disturbances` varaible.
    
    Parameters:
    ds (xr.DataArray): the dataset containing the biomass stocks
    growth_rate_map (xr.DataArray): the map of growth rates
    disturbances (xr.DataArray): the map of disturbances

    Returns:
    (xr.DataArray,xr.DataArray): the biomass stocks in dense forests for the 100 Mg/ha and 200 Mg/ha thresholds
    """
    

    # define the base map for the growth rates
    base_map = xr.zeros_like(ds.sel(time=ds['time'].values[0]))

    # reproject the growth rate map and disturbances to match the dataset
    gm = growth_rate_map.rio.reproject_match(ds)
    dist_map = disturbances.rio.reproject_match(ds)

    # fill in missing years in the disturbance map
    missing_years = list(set(ds['time'].values).difference(set(dist_map['time'].values)))
    if len(missing_years)>0:
        dist_map = xr.concat([dist_map,dist_map.interp(time=missing_years)],dim='time').sortby('time')
        dist_map = dist_map.bfill('time')

    # calculate the biomass accumulated in undisturbed dense forests
    intact_changes = xr.concat([base_map+gm*i for i in (ds['time']-ds['time'][0]).values],dim='time').transpose('landcover','time','y','x')
    intact_changes['time'] = ds['time']

    # apply the growth rates to the forest and nonforest fractions of the map based on their relative biomass contribution
    intact_changes = intact_changes*(ds/ds.sum(dim='landcover'))

    # remove from the "dense forests" the area fraction lost to disturbances
    intact_changes_disturbances = intact_changes.copy()
    intact_changes_disturbances[:,1:,:,:] = intact_changes_disturbances[:,1:,:,:]*(1-dist_map.fillna(0))

    # add the "dense forest" growth to the data
    merged_100  = (intact_changes_disturbances.where(ds.sum(dim='landcover')>100).fillna(0) + ds)
    merged_200  = (intact_changes_disturbances.where(ds.sum(dim='landcover')>200).fillna(0) + ds)
    
    # drop zero values
    merged_100 = merged_100.where(merged_100!=0)
    merged_200 = merged_200.where(merged_200!=0)
    
    return merged_100,merged_200



In [5]:
def analyze_dataset_new(ds:xr.DataArray, growth_rate_map:xr.DataArray, disturbances:xr.DataArray)->Tuple[xr.DataArray,xr.DataArray]:
    """
    This function calculates the total biomass stocks assuming that "dense forests" (defined according to two thresholds - 100 and 200 Mg/ha) are growing at a constant rate.
    The growth rates are defined in the `growth_rate_map` variable. 
    From the base growth rate, we remove area that was affected by disturbance, defined by the `disturbances` varaible.
    
    Parameters:
    ds (xr.DataArray): the dataset containing the biomass stocks
    growth_rate_map (xr.DataArray): the map of growth rates
    disturbances (xr.DataArray): the map of disturbances

    Returns:
    (xr.DataArray,xr.DataArray): the biomass stocks in dense forests for the 100 Mg/ha and 200 Mg/ha thresholds
    """
    

    base_map = xr.zeros_like(ds.sel(time=ds['time'].values[0]))

    # reproject the growth rate map and disturbances to match the dataset
    gm = growth_rate_map.rio.reproject_match(ds)
    dist_map = disturbances.rio.reproject_match(ds)

    # fill in missing years in the disturbance map
    missing_years = list(set(ds['time'].values).difference(set(dist_map['time'].values)))
    if len(missing_years)>0:
        dist_map = xr.concat([dist_map,dist_map.interp(time=missing_years)],dim='time').sortby('time')
        dist_map = dist_map.bfill('time')

    # calculate the biomass accumulated in undisturbed dense forests
    intact_changes = xr.concat([base_map+gm*i for i in (ds['time']-ds['time'][0]).values],dim='time').transpose('landcover','time','y','x')
    intact_changes['time'] = ds['time']

    # apply the growth rates to the forest and nonforest fractions of the map based on their relative biomass contribution
    intact_changes = intact_changes*(ds/ds.sum(dim='landcover'))

    # remove from the "dense forests" the area fraction lost to disturbances
    intact_changes_disturbances = intact_changes.copy()
    intact_changes_disturbances[:,1:,:,:] = intact_changes_disturbances[:,1:,:,:]*(1-dist_map.fillna(0))

    
    # sum the "dense" and "non dense" forest biomass
    merged_100 = intact_changes_disturbances.where(ds.sum(dim='landcover')>100)+ds
    merged_200 = intact_changes_disturbances.where(ds.sum(dim='landcover')>200)+ds

    
    return merged_100,merged_200

## 4. Run analysis

In [6]:
# convert total emissions (TgC/yr) to densities (MgC/ha/yr)
def convert_emissions_to_density(da):
     da = da*1e12/calc_pixel_area(da)/100
     da['time'] = xu_data['time'][1:]
     return da

# calculate the emission density for each type of disturbance
fire_density = convert_emissions_to_density(fire_data.fillna(0)+fire_nonforest_data.fillna(0))
deforestation_density = convert_emissions_to_density(deforestation_data)
degradation_density = convert_emissions_to_density(degradation_data)

# factors from Xu et al.
f_D = 0.07
f_B = 0.3
f_C = 1

# calculate the area affected according to the equation presented in the introduction
PDA = (degradation_density/xu_data/f_D).fillna(0)
PCA = (deforestation_density/xu_data/f_C).fillna(0)
PBA = (fire_density/xu_data/f_B).fillna(0)

# sum all disturbances and make sure they are not greater than 1
disturbances = PDA+PCA+PBA
disturbances = xr.where(disturbances>1,1,disturbances)
disturbances.rio.write_crs('EPSG:4326',inplace=True);

In [7]:
# find the files that need to be processed
files =pd.Series(glob('../results/02_convert_AGB_TB/*.nc'))
files = files[files.str.contains('xu|liu|chen|LVOD')  & ~files.str.contains('_100|_200')].values
for file in tqdm(files):
    # load file
    ds =xr.open_dataarray(file,decode_times=False)

    # analyze file
    ds_100,ds_200 = analyze_dataset(ds,growth_rate_map,disturbances)
    print(file)
    
    # save results
    ds_100.to_netcdf(file[:-3] + '_100.nc')
    ds_200.to_netcdf(file[:-3] + '_200.nc')

  0%|          | 0/30 [00:00<?, ?it/s]

../results/02_convert_AGB_TB/LVOD_xu_huang_TB.nc


  3%|▎         | 1/30 [00:03<01:29,  3.09s/it]

../results/02_convert_AGB_TB/LVOD_song_huang_TB.nc


  7%|▋         | 2/30 [00:06<01:26,  3.09s/it]

...

 93%|█████████▎| 28/30 [04:43<00:08,  4.04s/it]

../results/02_convert_AGB_TB/LVODmin_song_ma_TB.nc


 97%|█████████▋| 29/30 [04:45<00:03,  3.57s/it]

../results/02_convert_AGB_TB/LVODmin_CCI_ma_TB.nc


100%|██████████| 30/30 [04:48<00:00,  9.60s/it]
